In [ ]:
import gitlab
import re
import json
import pandas as pd
import time
token = ''

In [ ]:
gl = gitlab.Gitlab(url='http://localhost', private_token=token)

In [ ]:
df = pd.read_csv('cobol_pullrequests.csv')
# Pandas split column by delimiter
df['name'] = df['name'].astype(str)
df[['repo_owner', 'repo_name']] = df['name'].str.split('/', expand=True)
# Get title, body, head & base branch names
df['payload'] = df['payload'].apply(json.loads)

df['title'] = df.loc[:,'payload'].apply(lambda x: x.get('pull_request').get('title'))
df['body']  = df.loc[:,'payload'].apply(lambda x: x.get('pull_request').get('body'))
df['head']  = df.loc[:,'payload'].apply(lambda x: x.get('pull_request').get('head').get('label'))
df['base']  = df.loc[:,'payload'].apply(lambda x: x.get('pull_request').get('base').get('label'))
df = df[['login','name','head', 'base', 'title', 'body']]
df

In [ ]:

def replace_bot_substring(string):
    #replace [bot] for -bot
    result = re.sub(r"\[bot]", "-bot", string, re.IGNORECASE)
    return result

def create_user(gl, user_name):
    user_data = {'email': user_name+'@mail.com', 
                 'username': user_name, 
                 'name': user_name, 
                 'reset_password':False, 
                 'password':'password',
                 'skip_confirmation':True}
    gl.users.create(user_data)

def create_repo(gl, repo_name, repo_owner, project_name_with_namespace):
    project_data = {'name': repo_name, 
                    'visibility':'public', 
                    'initialize_with_readme':True}
    gl.projects.create(project_data, sudo=repo_owner)
    return gl.projects.get(project_name_with_namespace)

def create_fork(gl, project, user_name):
    #List all forks from project, get user_name and forked project id:
    fork_list =  [(fork.attributes.get('owner').get('username'), fork.attributes.get('id'))
                    for fork in project.forks.list(get_all=True)]
    
    fork_list_users = [item[0] for item in fork_list]
    
    # If user_name not in fork_list, fork it:
    if user_name not in fork_list_users:
        forked_project = project.forks.create({}, sudo=user_name)
        return gl.projects.get(id=forked_project.id)
    else:
        # Get index of forked project if exist:
        indices = [i for i, tupl in enumerate(fork_list) if tupl[0] == user_name]
        return gl.projects.get(id=fork_list[indices[0]][1])
    
def create_pull_request(project, head_branch, base_branch, title, body, tarject_project_id, user_name):
    try:
        project.mergerequests.create({'source_branch':head_branch,
                                      'target_branch':base_branch,
                                      'title':title,
                                      'body':body,
                                      'target_project_id':tarject_project_id}, sudo=user_name)
    except gitlab.exceptions.GitlabCreateError:
        attempt = 0
        while attempt < 100:
            try:
                branch_rename = f'{head_branch}_{len(project.branches.list(get_all=True))+attempt}'
                project.branches.create({'branch': branch_rename,'ref': 'master'}, sudo=user_name)
                project.mergerequests.create({'source_branch':branch_rename,
                                              'target_branch':base_branch,
                                              'title':title,
                                              'body':body,
                                              'target_project_id':tarject_project_id}, sudo=user_name)
                break
            except gitlab.exceptions.GitlabCreateError:
                attempt +=1

In [ ]:
def pull_request(i):
    
    user_name  = replace_bot_substring(df.iloc[i,:]['login'])
    repo_owner, repo_name = df.iloc[i,:]['name'].split('/')
    repo_owner, repo_name = replace_bot_substring(repo_owner),replace_bot_substring(repo_name)
    _, base_branch = df.iloc[i,:]['base'].split(':')
    _, head_branch = df.iloc[i,:]['head'].split(':')
    title = df.iloc[i,:]['title']
    body = df.iloc[i,:]['body']
    project_name_with_namespace =df.iloc[i,:]['name']
    
    print(f'{i} user:{user_name}, repo:{repo_owner}/{repo_name}, base: {base_branch}, head: {head_branch}')
    
    # 1. Create user if it does no exist:
    if not gl.users.list(search=user_name):
        create_user(gl, user_name)

    # 2. Create repo owner user if it does no exist:
    if not gl.users.list(search=repo_owner):
        create_user(gl, repo_owner)

    # 3. Create repo if it does not exist:
    try:
        #project_name_with_namespace = df.iloc[0,:]['name']
        project = gl.projects.get(project_name_with_namespace)
    except gitlab.exceptions.GitlabGetError:
        project = create_repo(gl, repo_name, repo_owner, project_name_with_namespace)

    tarject_project_id = project.id

    # 4. Create source (base) branch if it does not exist:
    try:
        time.sleep(0.5)
        project.branches.get(base_branch)
    except gitlab.exceptions.GitlabGetError:
        project.branches.create({'branch': base_branch,'ref': 'master'}, sudo=repo_owner)

    # 5. Fork project if user_name != repo_owner
    if user_name != repo_owner:
        forked_project = create_fork(gl, project, user_name)

        # 6. Create target (head) branch if it does not exist:
        try:
            time.sleep(0.5)
            forked_project.branches.get(head_branch)
        except gitlab.exceptions.GitlabGetError:
            forked_project.branches.create({'branch': head_branch,'ref': 'master'}, sudo=user_name)

        # 7. Create merge request (pull request):
        create_pull_request(forked_project, head_branch, base_branch, title, body, tarject_project_id, user_name)

    # if user_name == repo_owner, there is no need to fork
    else:
        # 6. Create target (head) branch if it does not exist:
        try:
            time.sleep(0.5)
            project.branches.get(head_branch)
        except gitlab.exceptions.GitlabGetError:
            project.branches.create({'branch': head_branch,'ref': 'master'}, sudo=user_name)

        # 7. Create merge request (pull request):
        create_pull_request(project, head_branch, base_branch, title, body, tarject_project_id, user_name)

In [ ]:
for i in df.index:
  
    attempt = 0
    while attempt < 10:
        attempt += 1
        print('attempt:', attempt)
        try:
            pull_request(i)
        except Exception as e:
            print(e)
            time.sleep(0.5)
            continue
        break

In [ ]:
# Remove all users and repo except Admin staff

users_ids = [user.id for user in gl.users.list(get_all=True) if user.id != 1]

for ids in users_ids:
    gl.users.delete(id=ids, hard_delete=True)

project_ids = [project.id for project in gl.projects.list(get_all=True) if project.id != 2]

for ids in project_ids:
    gl.projects.delete(id=ids)